In [2]:
import os
import pandas as pd

path = os.getcwd()

train_data = pd.read_csv("../../datasets/train.csv", index_col = 0, encoding = 'utf-8')

train_data.dropna(inplace = True)
train_data.drop(['감정_소분류','시스템문장1','시스템문장2','시스템문장3', '사람문장2', '사람문장3', '신체질환'], axis=1, inplace=True)
train_data.rename({'사람문장1':'text', '감정_대분류':'label'}, axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['label'])

X_train = train_data['text']
y_train = train_data['label']

X_train[5:10], y_train[5:10], y_train.value_counts()

(310    프리랜서로 일하면서 요즘 일이 없는데 가족들은 내가 열심히 하지 않은 탓이라고 해서...
 311     오늘 은퇴하고 왔는데 가족들이 그동안 고생했다며 나를 격려해주지 않아서 마음이 아팠어.
 312               앞으로 노후를 위해 준비할 것들이 많은데 의지할 사람은 없고 막막해.
 313         얼마 전 은행 계좌를 봤는데 생각보다 모아 둔 돈이 없더라고. 너무 충격이었어.
 314     그동안 담배를 너무 많이 피웠나 봐. 이제 와서 내가 모은 돈을 보니 너무 후회가 돼.
 Name: text, dtype: object,
 310    4
 311    4
 312    4
 313    4
 314    5
 Name: label, dtype: int32,
 label
 2    7679
 4    7671
 3    7661
 5    7564
 1    7424
 0    4695
 Name: count, dtype: int64)

In [4]:
# Load model directly
from transformers import AutoTokenizer, DistilBertModel

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
model = DistilBertModel.from_pretrained("distilbert/distilbert-base-multilingual-cased")

In [5]:
model.train()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [6]:
text = list(X_train)[:3]
encoded_input = tokenizer.batch_encode_plus(text, return_tensors='pt', padding=True)
output = model(**encoded_input)

In [7]:
encoded_input

{'input_ids': tensor([[   101,   9706,  40032,  18382,  10028,  27023,  14153,   9641, 119424,
          41850,   9632, 119362,  70146,   9089,  10739,  39218,  11664,   9952,
          25503,   9651,  21155,  10739,   9993,  11513,   8996,  12424,   9414,
          60469,  11513,   9322, 119118,  12965,    119,    102,      0],
        [   101,   9781,  17196,  11102, 119351,   9632, 119362,  14843,   8863,
          59894,   9545,  12310, 119424,  54141,  25503,   9531,  11467,    100,
           9266,  11664,   9408,   8863, 118728,  30936,   9379, 119170,  54141,
          59894,    119,   8932,  37712,  10739,    100,    119,    102],
        [   101,   9781,  17196,  11102, 119351,   9632, 119362,  14102,  11664,
           9965,  54141,  25503,   8924,  14153,   9251,  43739,  54780,   8863,
         118728,  21406,   8985,   9954,  71013,  11102,   9405,  61250,   9773,
          37568,  70146,   9425,  21614, 119424,  12965,    119,    102]]), 'attention_mask': tensor([[1, 1, 

In [11]:
output

BaseModelOutput(last_hidden_state=tensor([[[ 0.0864, -0.0049, -0.1367,  ...,  0.2188,  0.0968, -0.0684],
         [ 0.2903, -0.5238, -0.1615,  ..., -0.1489,  0.1981,  0.2658],
         [ 0.8242, -0.3524, -0.3196,  ...,  0.2147,  0.2687,  0.2642],
         ...,
         [ 0.2685, -0.2062, -0.3115,  ...,  0.4174,  0.2080,  0.0268],
         [ 0.3146, -0.3206, -0.1635,  ...,  0.3678,  0.2458, -0.0294],
         [ 0.3456, -0.2416, -0.0509,  ...,  0.4382,  0.2358,  0.0097]],

        [[ 0.1233, -0.0537, -0.0849,  ...,  0.1495,  0.1344, -0.0527],
         [-0.1766, -0.2488, -0.3115,  ...,  0.4022, -0.0260,  0.3715],
         [ 0.0786, -0.3074,  0.0587,  ...,  0.6179,  0.2107,  0.2963],
         ...,
         [ 0.4306, -0.2358, -0.1010,  ...,  0.4943,  0.1669, -0.1255],
         [ 0.4203, -0.2828, -0.2641,  ...,  0.2283,  0.1687,  0.0709],
         [ 0.4072, -0.3443,  0.1213,  ...,  0.1163,  0.1596,  0.0868]],

        [[ 0.1954,  0.0363, -0.0357,  ...,  0.2697,  0.0751, -0.0506],
         [-

In [10]:
output.last_hidden_state.shape

torch.Size([3, 35, 768])

In [ ]:
# last_hidden_state에 bert와 마찬가지로 classification head를 붙여서 학습시키면 될듯?